In [42]:
import os
import sys
import yaml
from typing import List, Dict, Tuple

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import infovar

sys.path.insert(0, os.path.join("..", "..", ".."))
from infobs.plots import Plotter

sys.path.insert(1, os.path.join("..", ".."))
from pdr_util import get_physical_env, simulate, latex_line, latex_param

results_path = os.path.join("results")
figures_path = os.path.join("map_figures")

plt.rc("text", usetex=True)

In [43]:
handler = infovar.ContinuousHandler()

handler.set_paths(
    save_path=results_path,
)

In [44]:
plotter = Plotter(
    line_formatter=latex_line,
    param_formatter=latex_param
)

latex_comb_lines = lambda ls: plotter.lines_comb_formatter(ls, short=True)
latex_comb_params = lambda ps: plotter.params_comb_formatter(ps)

In [45]:
lines_list = [
    # Simple lines
    ["co_v0_j1__v0_j0"],
    ["co_v0_j2__v0_j1"],
    ["co_v0_j3__v0_j2"],
    ["13c_o_j1__j0"],
    ["13c_o_j2__j1"],
    ["13c_o_j3__j2"],
    ["c_18o_j1__j0"],
    ["c_18o_j2__j1"],
    ["c_18o_j3__j2"],
    ["hcop_j1__j0"],
    ["hcop_j2__j1"],
    ["hcop_j3__j2"],
    ["hcop_j4__j3"],
    ["hnc_j1__j0"],
    ["hnc_j3__j2"],
    ["hcn_j1_f2__j0_f1"],
    ["hcn_j2_f3__j1_f2"],
    ["hcn_j3_f3__j2_f3"],
    ["cs_j2__j1"],
    ["cs_j3__j2"],
    ["cs_j5__j4"],
    ["cs_j6__j5"],
    ["cs_j7__j6"],
    # CN lines
    ["cn_n1_j0d5__n0_j0d5"],
    ["cn_n1_j1d5__n0_j0d5"],
    ["cn_n2_j1d5__n1_j0d5"],
    ["cn_n2_j2d5__n1_j1d5"],
    ["cn_n3_j3d5__n2_j2d5"],
    # C2H lines
    ["c2h_n1d0_j1d5_f2d0__n0d0_j0d5_f1d0"],
    ["c2h_n2d0_j2d5_f3d0__n1d0_j1d5_f2d0"],
    ["c2h_n3d0_j3d5_f4d0__n2d0_j2d5_f3d0"],
    ["c2h_n3d0_j2d5_f3d0__n2d0_j1d5_f2d0"],   
    ["c2h_n4d0_j4d5_f5d0__n3d0_j3d5_f4d0"],
    # Carbon lines
    ["c_el3p_j1__el3p_j0"],
    ["c_el3p_j2__el3p_j1"],
    ["cp_el2p_j3_2__el2p_j1_2"],
    # Some interesting combinations
    ["co_v0_j1__v0_j0", "co_v0_j2__v0_j1"],
    ["co_v0_j1__v0_j0", "co_v0_j2__v0_j1", "co_v0_j3__v0_j2"],
    #
    ["13c_o_j1__j0", "13c_o_j2__j1"],
    ["13c_o_j1__j0", "13c_o_j2__j1", "13c_o_j3__j2"],
    #
    ["c_18o_j1__j0", "c_18o_j2__j1"],
    ["c_18o_j1__j0", "c_18o_j2__j1", "c_18o_j3__j2"],
    #
    ["co_v0_j1__v0_j0", "13c_o_j1__j0"],
    ["co_v0_j1__v0_j0", "c_18o_j1__j0"],
    ["13c_o_j1__j0", "c_18o_j1__j0"],
    #
    ["c_el3p_j1__el3p_j0", "c_el3p_j2__el3p_j1"],
    ["c_el3p_j1__el3p_j0", "cp_el2p_j3_2__el2p_j1_2"],
    ["c_el3p_j2__el3p_j1", "cp_el2p_j3_2__el2p_j1_2"],
    #
    ["hcn_j1_f2__j0_f1", "hcn_j2_f3__j1_f2"],
    ["hcn_j1_f2__j0_f1", "hcn_j2_f3__j1_f2", "hcn_j3_f3__j2_f3"],
    #
    ["hcn_j1_f2__j0_f1", "hnc_j1__j0"],
    #
    ["cs_j2__j1", "cs_j3__j2"],
]
params = ["P"]
wins_features = ["P", "Avmax"]
stat = "mi"

In [46]:
ticksfontsize = 14
labelfontsize = 16
titlefontsize = 16

In [47]:
# Look for vmax
vmax = 0
for lines in lines_list:

    try:
        d = handler.read(
            lines, params, wins_features
        )
    except:
        continue

    data = d["stats"][stat]["data"]
    vmax = max(vmax, np.max(data))

    wins_features = d["features"]

In [48]:
_path = os.path.join(figures_path, "_".join(params))
if not os.path.isdir(_path):
    os.mkdir(_path)

_path = os.path.join(figures_path, "_".join(params), "_".join(wins_features))
if not os.path.isdir(_path):
    os.mkdir(_path)

In [49]:
for lines in tqdm(lines_list):

    try:
        d = handler.read(
            lines, params, wins_features
        )
    except:
        continue

    data = d["stats"][stat]["data"]
    samples = d["stats"][stat]["samples"]
    xticks, yticks = d["stats"][stat]["coords"]
    wins_features = d["features"]

    paramy, paramx = wins_features

    #

    fig = plt.figure(dpi=150)
    ax = plt.gca()

    X, Y = np.meshgrid(yticks, xticks)

    im = ax.pcolor(X, Y, data, cmap='jet', vmin=0, vmax=vmax)
    # ax.set_xlim(lims[params_regime[0]])
    # ax.set_ylim(lims[params_regime[1]])

    ax.set_xscale('log')
    ax.set_yscale('log')

    ax.set_xlabel(f"${plotter.param_formatter(paramx)}$", fontsize=labelfontsize)
    ax.set_ylabel(f"${plotter.param_formatter(paramy)}$", fontsize=labelfontsize)
    # ax.set_title(f"${plotter.lines_comb_formatter(lines)}$", fontsize=titlefontsize, pad=10)
    ax.set_title(f"Informativity on ${plotter.params_comb_formatter(params)}$", fontsize=titlefontsize, pad=10)

    cbar = fig.colorbar(im)
    cbar.set_label("Amount of information (bits)", labelpad=10)

    plt.savefig(os.path.join(figures_path, "_".join(params), "_".join(wins_features), f"{'_'.join(lines)}.png"))
    plt.close(fig)

100%|██████████| 52/52 [00:00<00:00, 2890.06it/s]
